In [ ]:
"""Workbook to quantify bias present in metadata
Q: Can you identify certain labels by using other metadata
e.g. find cell type using project+assay+other
"""
# pylint: disable=import-error, redefined-outer-name, use-dict-literal, too-many-lines, unused-import, unused-argument, pointless-statement

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from __future__ import annotations

from pathlib import Path
from typing import Any, Dict, List, Tuple

import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.svm import SVC

from epi_ml.utils.notebooks.paper.paper_utilities import (
    ASSAY,
    CELL_TYPE,
    LIFE_STAGE,
    SEX,
    MetadataHandler,
    SplitResultsHandler,
    create_mislabel_corrector,
)

In [4]:
base_dir = Path.home() / "Projects/epiclass/output/paper"
base_data_dir = base_dir / "data"
base_fig_dir = base_dir / "figures"
paper_dir = base_dir

if not base_fig_dir.exists():
    raise FileNotFoundError(f"Directory {base_fig_dir} does not exist.")

In [5]:
metadata_handler = MetadataHandler(paper_dir)
metadata_df = metadata_handler.load_metadata_df("v2")
metadata = metadata_handler.load_metadata("v2")

split_results_handler = SplitResultsHandler()

## Evaluate bias in input samples classification

### Collect observed average accuracy

In [6]:
results_dir = base_data_dir / "training_results" / "dfreeze_v2" / "hg38_100kb_all_none"

exclusion = ["cancer", "random", "track", "disease", "second", "end"]
exclude_names = ["chip", "no-mixed", "ct", "7c"]

all_split_results = split_results_handler.general_split_metrics(
    results_dir=results_dir,
    exclude_categories=exclusion,
    exclude_names=exclude_names,
    merge_assays=True,
    mislabel_corrections=create_mislabel_corrector(paper_dir),
    return_type="split_results",
)

In [ ]:
results_dir

In [7]:
concat_split_results: Dict[str, pd.DataFrame] = split_results_handler.concatenate_split_results(all_split_results, concat_first_level=True)  # type: ignore

In [8]:
for cat_name, df in list(concat_split_results.items()):
    new_df = metadata_handler.join_metadata(df, metadata)
    concat_split_results[cat_name] = new_df

In [9]:
avg_input_acc = {}
for cat_name, df in list(concat_split_results.items()):
    # filtered_df = df[df[ASSAY] == "input"]
    filtered_df = df
    acc = (filtered_df["True class"] == filtered_df["Predicted class"]).sum() / len(
        filtered_df
    )
    avg_input_acc[cat_name] = acc

In [ ]:
display(avg_input_acc)

In [11]:
avg_input_acc[SEX] = avg_input_acc["harmonized_donor_sex_w-mixed"]
concat_split_results[SEX] = concat_split_results["harmonized_donor_sex_w-mixed"]

avg_input_acc[ASSAY] = avg_input_acc["assay_epiclass_11c"]
concat_split_results[ASSAY] = concat_split_results["assay_epiclass_11c"]

### Compute max bias accuracy using metadata as input

In [13]:
def define_input_bias_categories(target_category: str) -> List[List[str]]:
    """Define bias categories used for bias analysis.

    Args:
        target_category (str): Classification target category. Is excluded from input lists.

    Returns:
        List[List[str]]: List of bias categories.
    """
    bias_categories_1 = [ASSAY, "project", "harmonized_biomaterial_type", CELL_TYPE]
    bias_categories_2 = [
        ASSAY,
        "project",
        "harmonized_biomaterial_type",
        CELL_TYPE,
        LIFE_STAGE,
    ]
    bias_categories_3 = [ASSAY, "project", "harmonized_biomaterial_type", CELL_TYPE, SEX]
    bias_categories_4 = [
        ASSAY,
        "project",
        "harmonized_biomaterial_type",
        CELL_TYPE,
        SEX,
        LIFE_STAGE,
    ]

    all_bias_categories = [
        bias_categories_1,
        bias_categories_2,
        bias_categories_3,
        bias_categories_4,
    ]
    for bias_categories in all_bias_categories:
        try:
            bias_categories.remove(target_category)
        except ValueError:
            pass
    return all_bias_categories

In [14]:
def create_models() -> List:
    """Create models for bias analysis."""
    lr_model_1 = LogisticRegression(
        solver="lbfgs", max_iter=1000, multi_class="multinomial", random_state=42
    )
    lr_model_2 = LogisticRegression(
        solver="lbfgs", max_iter=1000, multi_class="ovr", random_state=42
    )
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    svm_model = SVC(kernel="linear", random_state=42)
    svm_model_rbf = SVC(kernel="rbf", random_state=42)
    return [lr_model_1, lr_model_2, rf_model, svm_model, svm_model_rbf]

In [19]:
def filter_samples(
    metadata_df: pd.DataFrame, target_category: str, verbose: bool = True
) -> pd.DataFrame:
    """Filter samples based on the output category to match the original training set."""
    df = metadata_df.copy(deep=True)

    if "md5sum" not in df.columns:
        df["md5sum"] = df.index

    df = df[df["md5sum"].isin(concat_split_results[target_category]["md5sum"])]

    if verbose:
        print("Metadata shape:", metadata_df.shape)
        print("Filtered shape:", df.shape)
        display(df[target_category].value_counts())

    return df  # type: ignore


def find_max_bias(
    metadata_df: pd.DataFrame, target_category: str, verbose: bool = True
) -> Dict[Tuple[str, ...], float]:
    """Find the bias categories that provide the highest accuracy for the target category."""

    filtered_df = filter_samples(metadata_df, target_category)

    max_bias_dict = {}
    for bias_categories in define_input_bias_categories(target_category):
        print(f"Using bias categories: {bias_categories}")
        X = filtered_df[bias_categories]
        y = filtered_df[target_category]

        # one-hot encode the data
        X_encoded = OneHotEncoder().fit_transform(X).toarray()  # type: ignore
        y_encoded = LabelEncoder().fit_transform(y)

        max_acc = 0
        for model in create_models():
            scores = cross_val_score(
                model, X_encoded, y_encoded, cv=10, scoring="accuracy", n_jobs=-1
            )
            if verbose:
                print(f"Model: {model}")
                print(f"Accuracy: {np.mean(scores):.2f} (+/- {np.std(scores):.2f})")
            if np.mean(scores) > max_acc:
                max_acc = np.mean(scores)
                max_bias_dict[tuple(bias_categories)] = max_acc

    return max_bias_dict

In [ ]:
def compute_all_max_bias(
    metadata_df: pd.DataFrame, target_categories: List[str], verbose: bool = True
) -> Dict[str, Any]:
    """Compute the max metadata bias for all target categories."""
    final_results: Dict[str, Any] = {}
    for target_category in target_categories:
        if verbose:
            print(f"Target category: {target_category}")

        max_bias_dict = find_max_bias(metadata_df, target_category)
        max_bias_cats, max_bias_acc = max(max_bias_dict.items(), key=lambda x: x[1])
        if verbose:
            print(f"Max bias categories: {max_bias_cats}")
            print(f"Max bias acc: {max_bias_acc}")

        MLP_acc = avg_input_acc[target_category]

        acc_to_compare = [
            acc for cat, acc in avg_input_acc.items() if cat in max_bias_cats
        ]
        avg_MLP_acc = np.mean(acc_to_compare)
        max_acc_with_bias = max_bias_acc * avg_MLP_acc

        if verbose:
            print("CLASSIFICATION ACCURACY")
            print(f"Average {target_category} observed acc: {MLP_acc:.1%}")
            print(f"Average MLP acc on bias categories: {avg_MLP_acc:.1%}")
            print(
                f"Max avg acc with bias from ({max_bias_cats}): {max_acc_with_bias:.1%}"
            )
            print(f"Not accounted for: {MLP_acc - max_acc_with_bias:.1%}\n")

        final_results[target_category] = {
            "max_bias_cats": max_bias_cats,
            "max_bias_acc": max_bias_acc,
            "MLP_acc": MLP_acc,
            "bias_avg_MLP_acc": avg_MLP_acc,
            "max_bias_acc_corrected": max_acc_with_bias,
            "acc_diff": MLP_acc - max_acc_with_bias,
        }

    return final_results

In [ ]:
target_categories = ["project", "harmonized_biomaterial_type", CELL_TYPE, SEX, LIFE_STAGE]
final_results = compute_all_max_bias(metadata_df, target_categories)

final_results_df = pd.DataFrame.from_dict(final_results, orient="index")
final_results_df.to_csv("metadata_bias_analysis_results.csv")